In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.datasets import load_digits
from matplotlib import font_manager
from matplotlib import gridspec
from math import factorial
import sklearn
import pprint
import re
from sklearn.preprocessing import LabelEncoder
import time
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense


import warnings
warnings.filterwarnings('ignore')

font_fname = 'C:/Windows/Fonts/malgun.ttf'
font_family = font_manager.FontProperties(fname=font_fname).get_name()

plt.rcParams["font.family"] = font_family
plt.rcParams["axes.unicode_minus"] = False


%matplotlib inline

In [2]:
청약 = pd.read_csv('청약홈_분양정보_2020년9월.csv')

In [3]:
청약['타입'] = 청약['타입'].replace('APT', '아파트')

In [4]:
청약[['주택공급면적', '전용면적']].head()

,주택공급면적,전용면적
0,84.8474,059.9706A
1,85.5411,059.9702B
2,111.8544,084.9883A
3,112.1487,084.9939B
4,112.9441,084.8985C


In [5]:
청약['전용면적'] = 청약['전용면적'].str.extract(r'(\d+\.\d{1,2})').astype(float)

In [134]:
청약[['주택공급면적', '전용면적']]

,주택공급면적,전용면적
0,84.8474,59.97
1,85.5411,59.97
2,111.8544,84.98
3,112.1487,84.99
4,112.9441,84.89
...,...,...
828,111.8900,84.94
829,112.1300,84.91
830,131.0300,101.95
831,203.1700,152.71


In [136]:
청약.to_csv('청약홈_분양정보_2020년9월_수정.csv',encoding='cp949', index=False)

In [137]:
청약 = pd.read_csv('청약홈_분양정보_2020년9월_수정2.csv',encoding='cp949')

In [138]:
청약.to_csv('청약홈_분양정보_2020년9월_수정2.csv',encoding='cp949', index=False)

In [139]:
청약1 = pd.read_csv('청약홈_분양정보_2020년9월_수정2.csv', encoding='cp949')

In [140]:
청약2 = pd.read_csv('예지님_분양정보_데이터.csv', encoding='utf-8')

In [141]:
청약3 = pd.read_csv('수종님_202110_202209.csv', encoding='cp949')

In [226]:
청약4 = pd.read_csv('은옥님2_202210_202309.csv', encoding='utf-8')

In [143]:
청약1.keys()

Index(['타입', '아파트명', '면적', '법정동주소', '도로명주소', '위도', '경도', '세대수', '임대세대수',
       '영구임대세대수', '국민임대세대수', '민간임대세대수', '공공임대세대수', '장기임대세대수', '기타임대세대수', '최고층',
       '최저층', '사용승인일', '최대공급면적', '최소공급면적', '총아파트동수', '용적률', '건폐율', '주차대수',
       '세대평균_주차대수', '건설사', '주택공급면적', '전용면적', '전용율', '방수', '욕실수', '해당면적_세대수',
       '현관구조', '가격', '초등학교_학군정보', '초등학교_설립정보', '입주예정월', '공급금액(최고가 기준)'],
      dtype='object')

In [144]:
청약2.keys()

Index(['Unnamed: 0', '타입', '아파트명', '면적', '법정동주소', '도로명주소', '위도 구글 API',
       '경도 구글 API', '세대수', '임대세대수', '영구임대세대수', '국민임대세대수', '민간임대세대수', '공공임대세대수',
       '장기임대세대수', '기타임대세대수', '최고층', '최저층', '사용승인일', '최대공급면적', '최소공급면적',
       '총아파트동수', '매매', '전세', '월세', '단기', '부지의 총면적', '건축물의 총 평방 미터 면적',
       '건물의 바닥 면적', '용적률', '건폐율', '주차대수', '세대평균_주차대수', '건설사', '난방', '난방연료',
       '면적(참조)', '면적(수식)', '공급면적', '전용면적', '전용율', '방수', '욕실수', '해당면적_세대수',
       '현관구조', '재산세', '재산세합계', '지방교육세', '재산세_도시지역분', '종합부동산세', '결정세액',
       '농어촌특별세', '가격', '겨울관리비', '여름관리비', '매매호가', '전세호가', '월세호가', '실거래가',
       '초등학교_학군정보', '초등학교_설립정보', '초등학교_남학생수', '초등학교_여학생수', '입주예정월',
       '공급금액(최고가 기준)'],
      dtype='object')

In [232]:
청약1_수정 = 청약1.drop(['타입','사용승인일'], axis=1)
청약1_수정.rename(columns={'주택공급면적': '공급면적', '입주예정월':'입주예정일', '공급금액(최고가 기준)': '공급액(만원)'}, inplace=True)

In [233]:
청약2_수정 = 청약2.drop(['Unnamed: 0', '타입', '매매', '전세', '월세', '단기', '부지의 총면적', '건축물의 총 평방 미터 면적',
       '건물의 바닥 면적', '난방', '난방연료','면적(참조)', '면적(수식)','재산세', '재산세합계', '지방교육세', '재산세_도시지역분', '종합부동산세', '결정세액',
       '농어촌특별세','겨울관리비', '여름관리비', '매매호가', '전세호가', '월세호가', '실거래가','초등학교_남학생수', '초등학교_여학생수', '사용승인일'],axis=1)

청약2_수정.rename(columns={'위도 구글 API': '위도', '경도 구글 API': '경도', '입주예정월':'입주예정일', '공급금액(최고가 기준)': '공급액(만원)'}, inplace=True)

In [234]:
청약3_수정 =  청약3.drop('Unnamed: 0',axis=1)

In [235]:
청약4_수정 =  청약4.drop(['타입','사용승인일'],axis=1)

In [236]:
tup_list = []
for i in range(len(청약4_수정.keys())):
    tup_list.append((청약1_수정.keys()[i], 청약2_수정.keys()[i], 청약3_수정.keys()[i], 청약4_수정.keys()[i]))

In [238]:
# 컬럼명 확인 및 비교
columns_df1 = set(청약1_수정.columns)
columns_df2 = set(청약2_수정.columns)
columns_df3 = set(청약3_수정.columns)
columns_df4 = set(청약4_수정.columns)

if columns_df1 == columns_df2 == columns_df3 == columns_df4:
    print("모든 데이터프레임의 컬럼이 동일합니다.")
else:
    print("데이터프레임의 컬럼이 다를 수 있습니다.")

모든 데이터프레임의 컬럼이 동일합니다.


In [246]:
frames = [청약1_수정, 청약2_수정, 청약3_수정, 청약4_수정]
청약_합본 = pd.concat(frames).reset_index(drop=True)
청약_합본.dropna(subset=['위도', '경도', '공급액(만원)'], inplace=True)

In [247]:
청약_합본.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3347 entries, 0 to 3365
Data columns (total 36 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   아파트명       3347 non-null   object 
 1   면적         3347 non-null   object 
 2   법정동주소      3345 non-null   object 
 3   도로명주소      1976 non-null   object 
 4   위도         3347 non-null   float64
 5   경도         3347 non-null   float64
 6   세대수        3347 non-null   object 
 7   임대세대수      3108 non-null   object 
 8   영구임대세대수    3059 non-null   float64
 9   국민임대세대수    3059 non-null   float64
 10  민간임대세대수    3059 non-null   float64
 11  공공임대세대수    3059 non-null   float64
 12  장기임대세대수    3059 non-null   float64
 13  기타임대세대수    3058 non-null   float64
 14  최고층        3302 non-null   float64
 15  최저층        3288 non-null   float64
 16  최대공급면적     3310 non-null   float64
 17  최소공급면적     3310 non-null   float64
 18  총아파트동수     3319 non-null   float64
 19  용적률        3227 non-null   object 
 20  건폐율     

 6   세대수        3349 non-null   object  문자열 콤마
 7   임대세대수      3108 non-null   object  nan 문자열 콤마
 19  용적률        3229 non-null   object  nan #DIV/0! 문자열 %
 20  건폐율        3203 non-null   object  nan #DIV/0! 문자열 %
 21  주차대수       3171 non-null   object nan 문자열 콤마
 26  전용율        3346 non-null   object  nan %
 29  해당면적_세대수   3339 non-null   object nan 문자열 콤마
 34  입주예정일      3268 non-null   object  nan 시계열통일

In [302]:
청약_합본['세대수'].unique()

array([878, 1466, 160, 299, 1223, 1388, 188, 672, 99, 1048, 48, 42, 489,
       1235, 218, 1163, 243, 626, 1529, 280, 58, 407, 576, 1772, 950, 45,
       877, 583, 420, 375, 656, 824, 1050, 1516, 170, 424, 826, 1086, 999,
       544, 168, 665, 186, 393, 820, 1509, 565, 230, 606, 438, 233, 64,
       172, 767, 700, 605, 32, 136, 1187, 1130, 3432, 766, 1246, 225, 873,
       1108, 610, 935, 504, 4774, 348, 664, 1140, 3603, 475, 647, 87, 930,
       604, 653, 747, 1714, 860, 155, 666, 79, 702, 318, 466, 144, 1297,
       620, 1228, 804, 1335, 1872, 680, 428, 517, 512, 380, 677, 920, 412,
       2426, 1559, 1100, 1218, 2054, 303, 909, 358, 726, 345, 745, 1406,
       1834, 1205, 722, 1409, 385, 437, 534, 499, 413, 178, 294, 346, 342,
       430, 2379, 35, 55, 37, 1926, 353, 642, 609, 602, 496, 139, 207,
       435, 679, 584, 740, 841, 1134, 43, 254, 336, 394, 1282, 627, 976,
       1378, 174, 640, 986, 550, 1696, 559, 341, 2339, 809, 660, 130, 90,
       360, 27, 56, 556, 40, 808, 707, 444

In [309]:
청약_합본_copy = 청약_합본.copy()
청약_합본_copy['세대수'] = 청약_합본_copy['세대수'].astype(str).str.replace(',', '').fillna(-1).astype(int)

청약_합본_copy['임대세대수'] = 청약_합본_copy['임대세대수'].astype(str).str.replace(',', '').fillna(-1).astype(float)

청약_합본_copy['용적률'] = 청약_합본_copy['용적률'].astype(str).str.replace('%', '').fillna(-1).replace('#DIV/0!', -1).astype(float)

청약_합본_copy['건폐율'] = 청약_합본_copy['건폐율'].astype(str).str.replace('%', '').fillna(-1).replace('#DIV/0!', -1).astype(float)

청약_합본_copy['주차대수'] = 청약_합본_copy['주차대수'].astype(str).str.replace(',', '').fillna(-1).astype(float)

청약_합본_copy['전용율'] = 청약_합본_copy['전용율'].astype(str).str.replace('%', '').fillna(-1).astype(float)

청약_합본_copy['해당면적_세대수'] = 청약_합본_copy['해당면적_세대수'].astype(str).str.replace(',', '').fillna(-1).astype(float)


In [304]:
청약_합본_copy['세대수']

0       878
1       878
2       878
3       878
4       878
       ... 
3361     94
3362     94
3363     94
3364     94
3365     94
Name: 세대수, Length: 3347, dtype: int32

In [291]:
청약_합본['세대수'].isna().sum()

0

In [312]:
청약_합본_copy.to_csv('청약홈_합본.csv', encoding='cp949')

In [155]:
병원 = pd.read_csv('fulldata_01_01_01_P_병원.csv', encoding='cp949')

In [157]:
병원.keys()

Index(['번호', '개방서비스명', '개방서비스아이디', '개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자',
       '영업상태구분코드', '영업상태명', '상세영업상태코드', '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자',
       '재개업일자', '소재지전화', '소재지면적', '소재지우편번호', '소재지전체주소', '도로명전체주소', '도로명우편번호',
       '사업장명', '최종수정시점', '데이터갱신구분', '데이터갱신일자', '업태구분명', '좌표정보(x)', '좌표정보(y)',
       '의료기관종별명', '의료인수', '입원실수', '병상수', '총면적', '진료과목내용', '진료과목내용명', '지정취소일자',
       '완화의료지정형태', '완화의료담당부서명', '구급차특수', '구급차일반', '총인원', '구조사수', '허가병상수',
       '최초지정일자', '관리주체', 'Unnamed: 45'],
      dtype='object')

In [158]:
병원['업태구분명'].unique()

array(['요양병원(일반요양병원)', '정신병원', '병원', '요양병원(노인병원)', '종합병원', '한방병원', '치과병원',
       '요양병원(정신병원)', nan], dtype=object)

In [164]:
의원 = pd.read_csv('fulldata_01_01_02_P_의원.csv', encoding='cp949', error_bad_lines=False)

Skipping line 96565: expected 45 fields, saw 46



In [166]:
의원.keys()

Index(['번호', '개방서비스명', '개방서비스아이디', '개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자',
       '영업상태구분코드', '영업상태명', '상세영업상태코드', '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자',
       '재개업일자', '소재지전화', '소재지면적', '소재지우편번호', '소재지전체주소', '도로명전체주소', '도로명우편번호',
       '사업장명', '최종수정시점', '데이터갱신구분', '데이터갱신일자', '업태구분명', '좌표정보(x)', '좌표정보(y)',
       '의료기관종별명', '의료인수', '입원실수', '병상수', '총면적', '진료과목내용', '진료과목내용명', '지정취소일자',
       '완화의료지정형태', '완화의료담당부서명', '구급차특수', '구급차일반', '총인원', '구조사수', '허가병상수',
       '최초지정일자', 'Unnamed: 44'],
      dtype='object')

In [176]:
의원[['좌표정보(x)', '좌표정보(y)']].head()

,좌표정보(x),좌표정보(y)
0,192630.735112,185314.617632
1,190646.777107,189589.427851
2,190645.299575,189353.478160
3,188411.596132,190307.223169
4,190524.805310,190074.515460


In [181]:
from pyproj import Transformer

# 좌표 변환에 사용할 변환 객체 생성
transformer = Transformer.from_crs("EPSG:2097", "EPSG:4326")  # TM -> WGS84


# 좌표 변환
의원['위도'], 의원['경도'] = transformer.transform(의원['좌표정보(x)'], 의원['좌표정보(y)'])
병원['위도'], 병원['경도'] = transformer.transform(병원['좌표정보(x)'], 병원['좌표정보(y)'])

In [183]:
의원['업태구분명'].unique()

array(['치과의원', '한의원', '의원', '보건진료소', '보건지소', '보건소', '조산원', '보건의료원'],
      dtype=object)

In [190]:
의원['업태구분명'].value_counts()

의원       59482
치과의원     27444
한의원      25398
보건진료소     1930
보건지소      1356
보건소        254
조산원        105
보건의료원       15
Name: 업태구분명, dtype: int64

In [186]:
의원['영업상태명'].unique()

array(['영업/정상', '취소/말소/만료/정지/중지', '폐업', '휴업'], dtype=object)

In [191]:
의원['영업상태명'].value_counts()

영업/정상             72239
폐업                43125
취소/말소/만료/정지/중지      445
휴업                  175
Name: 영업상태명, dtype: int64

In [195]:
의원_선택 = 의원[의원['영업상태명'] == '영업/정상']

In [203]:
의원_선택.dropna(subset=['위도', '경도'], inplace=True)

In [204]:
의원_선택.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71052 entries, 0 to 72238
Data columns (total 47 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   번호           71052 non-null  int64  
 1   개방서비스명       71052 non-null  object 
 2   개방서비스아이디     71052 non-null  object 
 3   개방자치단체코드     71052 non-null  int64  
 4   관리번호         71052 non-null  object 
 5   인허가일자        71052 non-null  object 
 6   인허가취소일자      0 non-null      float64
 7   영업상태구분코드     71052 non-null  int64  
 8   영업상태명        71052 non-null  object 
 9   상세영업상태코드     71052 non-null  int64  
 10  상세영업상태명      71052 non-null  object 
 11  폐업일자         0 non-null      object 
 12  휴업시작일자       4 non-null      object 
 13  휴업종료일자       83 non-null     object 
 14  재개업일자        0 non-null      float64
 15  소재지전화        70509 non-null  object 
 16  소재지면적        0 non-null      float64
 17  소재지우편번호      25557 non-null  object 
 18  소재지전체주소      58540 non-null  object 
 19  도로명전

In [189]:
병원['업태구분명'].unique()

array(['요양병원(일반요양병원)', '정신병원', '병원', '요양병원(노인병원)', '종합병원', '한방병원', '치과병원',
       '요양병원(정신병원)', nan], dtype=object)

In [192]:
병원['업태구분명'].value_counts()

병원              2954
요양병원(일반요양병원)    2639
한방병원            1211
종합병원             518
치과병원             414
정신병원             363
요양병원(노인병원)        72
요양병원(정신병원)        21
Name: 업태구분명, dtype: int64

In [187]:
병원['영업상태명'].unique()

array(['영업/정상', '취소/말소/만료/정지/중지', '폐업', '휴업'], dtype=object)

In [193]:
병원['영업상태명'].value_counts()

영업/정상             5177
폐업                2776
취소/말소/만료/정지/중지     147
휴업                  95
Name: 영업상태명, dtype: int64

In [198]:
병원_선택 = 병원[병원['영업상태명'] == '영업/정상']
병원_선택 = 병원_선택[(병원_선택['업태구분명'] == '병원') | (병원_선택['업태구분명'] == '종합병원')]

In [201]:
병원_선택.dropna(subset=['위도', '경도'], inplace=True)

In [205]:
병원_선택.to_csv('fulldata_01_01_01_P_병원_수정.csv', encoding='cp949')
의원_선택.to_csv('fulldata_01_01_02_P_의원_수정.csv', encoding='cp949')

In [3]:
청약_합본_copy = pd.read_csv('청약홈_합본.csv', encoding='cp949')
병원_선택 = pd.read_csv('fulldata_01_01_01_P_병원_수정.csv', encoding='cp949')
의원_선택 = pd.read_csv('fulldata_01_01_02_P_의원_수정.csv', encoding='cp949')

In [7]:
청약_합본_copy[['위도','경도']].head()

,위도,경도
0,37.537863,127.062859
1,37.537863,127.062859
2,37.537863,127.062859
3,37.537863,127.062859
4,37.537863,127.062859


In [8]:
병원_선택[['위도','경도']].head()

,위도,경도
0,36.522615,127.300884
1,35.684853,128.672944
2,36.187347,126.552156
3,36.186740,126.557533
4,36.195495,126.733553


In [9]:
의원_선택[['위도','경도']].head()

,위도,경도
0,35.232831,126.836593
1,35.215002,126.883578
2,35.214987,126.880987
3,35.194863,126.891488
4,35.213908,126.888908


In [4]:
청약_합본_copy['위경도'] = list(zip(청약_합본_copy['위도'], 청약_합본_copy['경도']))
병원_선택['위경도'] = list(zip(병원_선택['위도'], 병원_선택['경도']))
의원_선택['위경도'] = list(zip(의원_선택['위도'], 의원_선택['경도']))

In [20]:
from haversine import haversine

병원수_리스트 = []

for 주택 in 청약_합본_copy['위경도']:
    갯수 = 0
    for 병원 in 병원_선택['위경도']:
        거리 = haversine(주택, 병원, unit = 'km')
        if 거리 <= 5:
            갯수 += 1 
    병원수_리스트.append(갯수)

청약_합본_copy['병원수'] = 병원수_리스트

In [21]:
청약_합본_copy['병원수'].value_counts()

0     3173
1       94
3       34
7       12
37      10
2        9
5        5
50       5
21       3
9        2
Name: 병원수, dtype: int64

In [23]:
의원수_리스트 = []

for 주택 in 청약_합본_copy['위경도']:
    갯수 = 0
    for 의원 in 의원_선택['위경도']:
        거리 = haversine(주택, 의원, unit = 'km')
        if 거리 <= 1:
            갯수 += 1 
    의원수_리스트.append(갯수)

청약_합본_copy['의원수'] = 의원수_리스트

In [24]:
청약_합본_copy['의원수'].value_counts()

0     3271
1       32
2       19
39      10
7        7
72       5
13       3
Name: 의원수, dtype: int64

In [25]:
의원_선택.shape

(71052, 49)

In [208]:
청약1_수정['건설사'].unique()

array(['롯데건설(주)', '에스케이건설(주),HDC현대산업개발(주)', '현대엔지니어링', '(주)동양건설산업',
       '지에스건설(주)', '(주)원건설', '㈜GS건설', '(주)성우에스아이', '삼성물산(주)',
       '주식회사스위트드림종합건설', '주식회사 덕인컨스텍', '㈜대우건설', '현대건설(주)', '㈜롯데건설',
       '혜림건설(주)', '중흥토건(주)', '한신공영(주)', '(주)한울건설산업', '(주)호반건설',
       'HDC현대산업개발(주)', 'GS건설(주)', '우석종합건설(주)', '제일건설(주)', '대림산업(주)',
       '일신건영(주)', '주식회사 금강주택', '(주)대원', '(주)라인건설', 'DL건설(주),DL이앤씨(주)',
       '현진에버빌(주)', '(주)대광건영', '(주)호반산업', '대방건설(주)', '(주)일군토건',
       '에이치디씨아이앤콘스(주)', '지에스건설주식회사', '(주)대우건설',
       '대림산업(주),금호산업(주),계룡건설산업(주),(주)고덕종합건설,(주)금성백조건설', '(주)한화건설',
       '(주)까뮤이앤씨', '(주)하호종합건설', '신한종합건설(주)', '금호건설(주)', '나경종합건설 주식회사',
       '신동아건설(주)', '(주)케이씨씨건설', 'HDC현대산업개발', '에쓰와이이앤씨(주)', '쌍용건설(주)',
       '(주)한라', '대우산업개발(주)', '(주)금강주택', '(주)대우건설,금호산업(주),(주)태영건설',
       'GS건설(주),(주)대우건설', '삼부토건(주)', 'HDC현대산업개발(주),롯데건설(주)', '(주)유림E&C',
       '(주)대우건설,에스케이건설(주)', '양우건설(주)', '천명토건(주) 외 1', '경남기업(주),(주)티케이케미칼',
       '(주)영무건설,(주)하가건설', '승윤종합건설(주),(주)길성종합건설', '포스코건설', '진산건설(주

In [209]:
청약4_수정['건설사'].unique()

array(['에스케이에코플랜트㈜', '삼성물산㈜', '(주)서희건설', '리센종합건설㈜', '롯데건설㈜', '중앙건설 주식회사',
       '(주)호반건설', '㈜한화', '경남기업 ㈜', '진흥기업(주)', 'HDC현대산업개발㈜',
       '우미건설(주), (주)부원건설', '보훈종합건설㈜', '(주)대우건설', '(주)로제비앙건설, 혜림건설㈜',
       '디엘이앤씨(주), 계룡건설산업(주)\t', '제일건설 주식회사', '에이치디씨현대산업개발', '두산건설㈜',
       '남광토건 주식회사', '주식회사씨디종합건설76.9500', '주식회사씨디종합건설', '청광건설주식회사',
       '지에스건설(주)\t', '(주)대우건설, GS건설(주), 롯데건설㈜', '(주)성호건설', '계룡건설산업㈜',
       '(주)대원, (주)대원에스앤디', '금호건설㈜', '(주)금강주택', '(주)포스코건설',
       '(주)우방,SM하이플러스㈜', '현대건설㈜', '혜림건설㈜', '지에스건설 주식회사', '동부건설㈜',
       '지에스건설㈜', '현대건설㈜, HDC현대산업개발㈜, ㈜대우건설, 롯데건설㈜', 'SK에코플랜트(주), 롯데건설㈜',
       '주)서희건설', '주식회사 창하종합건설', '중앙건설(주), 동우건설㈜',
       '지에스건설(주),(주)포스코이앤씨,(주)한화', '효성중공업㈜', '디엘이앤씨', '중흥토건㈜',
       '주식회사 금강주택', '대우산업개발주식회사', '대방건설㈜', 'GS건설㈜', '디엘이앤씨(주), 계룡건설산업㈜',
       '현대엔지니어링 주식회사', '주식회사 대광건영', '디엘이앤씨 주식회사, 코오롱글로벌 ㈜', '이수건설 주식회사',
       '(주)한양', '(주)태영건설, 금호건설(주), 신동아건설(주), (주)서영산업개발', '(주)호반산업',
       '디엘건설 주식회사, (주)대림', '에이치엘디앤아이한라 주식회사', '동부건설(주), 대보건설㈜',
  